# Auction

Objective: match limit and market orders together in a single batch auction.

## Order types

There are three order types: limit, market, and passive. Each order can be a bid (buy) or ask (sell). The direction is part of the storage key, so not included in the `Order` struct.

Note: the `amount` field is always measured **in the base asset**. In a previous version of our DEX smart contract, the `amount` field for market bids is in the quote asset. This is confusing and complicates the contract logic.

```rust
enum Order {
    Limit(LimitOrder),
    Market(MarketOrder),
    Passive(PassiveOrder),
}

struct LimitOrder {
    pub user: Addr,
    pub price: NonZero<Udec128_24>,
    pub amount: NonZero<Uint128>, // amount in the base asset
    pub remaining: Udec128_6,
    pub created_at_block_height: u64,
}

struct MarketOrder {
    pub user: Addr,
    pub amount: NonZero<Uint128>, // amount in the base asset
    pub remaining: Udec128_6,
    pub max_slippage: Udec128_24,
}

struct PassiveOrder {
    pub price: NonZero<Udec128_24>,
    pub amount: NonZero<Uint128>, // amount in the base asset
    pub remaining: Udec128_6,
}
```

## Order creation

- To create a **limit bid**, the user must send quote asset of amount `ceil(amount * price)`.
- To create a **limit ask**, the user must send base asset of amount `amount`.
- To create a **market bid**, the user must send quote asset of amount `ceil(amount * cutoff_price)`, where `cutoff_price` = `(1 + max_slippage) * best_ask_price`, where `best_ask_price` is the lowest ask price available in the resting order book. Resting order book means the order book at the end of last block, after the last block's auction, before any order creation or cancelation of this block has been executed.
- To create a **market ask**, the user must send base asset of amount `amount`.

## Order matching

At the end of the previous block, after the auction, we have saved in contract storage: 1) the best (highest) bid price, 2) the best (lowest) ask price, and 3) the mid price in the resting order book. The best bid/ask prices may be unavailable (`None`) if that side of the order book is empty.  `mid_price` is the arithmetic mean of the best bid and ask prices; if one side of the order book is empty, then take the best price of the other (non-empty) side; if both sides are empty, mid price is `None`.

At the end of the current block, we first load the best and mid prices from storage; then, we perform a batch auction involving 1) the resting order book, 2) limit and market orders received during the current block, and 3) orders from the passive liquidity pool, adjusted by the oracle prices updated in the current block.

Each market order is treated as a limit order:

- For market asks, the "limit price" is `(1 - max_slippage) * best_bid_price`.
- For market bids, the "limit price" is `(1 + max_slippage) * best_ask_price`.

If the best price is not available (meaning, that side of the order book is empty), the market order is canceled. (This should be extremely unlikely, as we always have the passive liquidity pool as the backstop liquidity.)

The auction should find a range of prices that maximizes the trading volume (measured in the base asset). We choose a clearing price within this range as follows:

- if `mid_price` is available, and:
  - it's within the range, then use `mid_price`;
  - it's bigger than the range's upper bound, use the upper bound;
  - it's smaller than the range's lower bound, use the lower bound;
- otherwise (`mid_price` is `None`), use the mid point of the range (i.e. the arithmetic mean between the upper and lower bounds).

In [ ]:

class OrderIterator:
    """
    An iterator that walks through one side of the order book, both limit and market orders, from the best to worst price.
    For this prototype, we ignore passive orders, since they are effectively the same as a limit order.
    """

    def __init__(self, limit_orders, market_orders, is_bid):
        self.limit_orders = limit_orders
        self.market_orders = market_orders
        self.is_bid = is_bid  # true = bid, false = ask

        self.i = 0  # index for limit orders
        self.j = 0  # index for market orders

    def __next_limit(self):
        i = self.i
        self.i += 1
        return self.limit_orders[i]

    def __next_market(self):
        j = self.j
        self.j += 1
        return self.market_orders[j]

    def next(self):
        # If we ran out of both order types, return `None`.
        if self.i >= len(self.limit_orders) and self.j >= len(self.market_order):
            return None;

        # If we ran out of limit orders but not market orders, return the market order.
        if self.i >= len(self.limit_orders):
            return self.__next_market()

        # If we ran out of market orders but not limit orders, return the limit order.
        if self.j >= len(self.market_orders):
            return self.__next_limit()

        # Neither order types has ran out. We choose one that has the better price.
        # For bids, better means higher; for asks, better means lower.
        # If they are the same price, we should choose the older one (the one that was created earlier,
        # or in our Rust implementation, the one that has a numerically smaller `order_id`).
        # However, in this prototype, for simplicity, we just return the market order.
        if self.is_bid:
            if self.limit_orders[self.i].price > self.market_orders[self.j].price:
                return self.__next_limit()
            else:
                return self.__next_market()
        else:
            if self.limit_orders[self.i].price < self.market_orders[self.j].price:
                return self.__next_limit()
            else:
                return self.__next_market()

In [ ]:
def do_auction(resting_order_book, limit_bids, limit_asks, market_bids, market_asks):
    """
    Perform the batch auction. Return the clearing price and volume measured in the base asset.

    Parameters:
    
    - `resting_order_book`: contains three fields: best (highest) bid price, best (lowest) ask price, and mid price of the resting order book;
      that is, the order book at the end of last block, after the last block's auction.
    - `limit_{bids,asks}`: are the limit order book with limit orders received during this block already merged in,
      ordered by price-time priority.
    - `market_{bids,asks}`: are market orders received during this block, ordered by price-time priority.

    Returns:

    - `clearing_price`: the single clearing price for all market and limit orders in this auction.
    - `volume`: the trading volume, measured in the base asset.
    - `bids_matched`: list of bid orders that have found a match.
    - `asks_matched`: list of ask orders that have found a match.
    """

    # Define the "limit" price of market bids, based on the best ask price in the resting order book.
    # If there's no ask orders in the resting order book, skip all market bids.
    if resting_order_book.best_ask_price is None:
        market_bids = []
    else:
        for market_bid in market_bids:
            market_bid.price = (1 + market_bid.max_slippage) * resting_order_book.best_ask_price

    # Same for market asks.
    if resting_order_book.best_bid_price is None:
        market_asks = []
    else:
        for market_ask in market_asks:
            market_ask.price = (1 - market_ask.max_slippage) * resting_order_book.best_bid_price

    # Combine the limit and market orders.
    bids = OrderIterator(limit_bids, market_bids, True)
    asks = OrderIterator(limit_asks, market_asks, True)

    # Starting condition for the loop.
    bid = bids.next()
    bid_is_new = True
    bid_volume = 0
    bids_matched = []
    
    ask = asks.next()
    ask_is_new = True
    ask_volume = 0
    asks_matched = []
    
    bounds = None # in Rust code we name this `range`, but it's a Python keyword...

    # Loop the orders and find the price range that maximizes trading volume.
    while True:
        if bid is None:
            break

        if ask is None:
            break

        if bid.price < ask.price:
            break

        bounds = (ask.price, bid.price)

        if bid_is_new:
            bid_volume += bid.amount # should be `bid.remaining`, but in this prototype we assume all orders are new (zero filled)
            bids_matched.append(bid)

        if ask_is_new:
            ask_volume += ask.amount
            asks_matched.append(ask)

        if bid_volume <= ask_volume:
            bid = bids.next()
            bid_is_new = True
        else:
            bid_is_new = False

        if ask_volume <= bid_volume:
            ask = asks.next()
            ask_is_new = True
        else:
            ask_is_new = False

    # Choose the clearing price.
    if bounds is None:
        clearing_price = None  # No match found.
    else:
        (lower, upper) = bounds
        if resting_order_book.mid_price is None:
            clearing_price = (lower + upper) / 2
        elif resting_order_book.mid_price < lower:
            clearing_price = lower
        elif resting_order_book.mid_price <= upper:
            clearing_price = resting_order_book.mid_price
        else:
            clearing_price = upper

    return clearing_price, volume, bids_matched, asks_matched